In [ ]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
%matplotlib inline
import sys 
sys.path.append("../..")

import pykep as pk
import paseos
from paseos import ActorBuilder, SpacecraftActor
from paseos.utils.load_default_cfg import load_default_cfg
import asyncio
paseos.set_log_level("DEBUG")

In [ ]:
from get_constellation import get_constellation
altitude = 400 * 1000
inclination = 40.0
nPlanes = 2
nSats = 1
t0 = pk.epoch(0)

planet_list,sats_pos_and_v = get_constellation(altitude,inclination,nSats,nPlanes,t0)

In [ ]:
fig = plt.figure(figsize=(6,6),dpi=100)
ax = plt.axes(projection='3d')
for i in range (nPlanes*nSats):
    pk.orbit_plots.plot_planet(planet_list[i],axes=ax,s=64)

In [ ]:
paseos_instances = []
earth = pk.planet.jpl_lp("earth")
for idx,sat_pos_v in enumerate(sats_pos_and_v):
    pos,v = sat_pos_v
    sat_actor = ActorBuilder.get_actor_scaffold(name="Sat_"+str(idx),
                                                actor_type=SpacecraftActor,
                                                epoch=t0)
    ActorBuilder.set_orbit(actor=sat_actor,position=pos,velocity=v,epoch=t0,central_body=earth)
    ActorBuilder.add_comm_device(actor=sat_actor,device_name="Link1",bandwidth_in_kbps=1000)
    ActorBuilder.set_power_devices(actor=sat_actor,battery_level_in_Ws=100,
                                   max_battery_level_in_Ws=1000,charging_rate_in_W=10)
    ActorBuilder.set_thermal_model(
            actor=sat_actor,
            actor_mass=50.0,
            actor_initial_temperature_in_K=273.15,
            actor_sun_absorptance=1.0,
            actor_infrared_absorptance=1.0,
            actor_sun_facing_area=1.0,
            actor_central_body_facing_area=1.0,
            actor_emissive_area=1.0,
            actor_thermal_capacity=1000
    )
    instance = paseos.init_sim(local_actor=sat_actor)
    paseos_instances.append(instance)